In [41]:
import pandas as pd
import glob
import regex as re
import querygen as qg

In [42]:
files = glob.glob("results/*.csv")
dfs = [pd.read_csv(f, index_col=0) for f in files]
files

['results/eval_results_BTO_oneshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_DBpedia_oneshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv']

In [43]:
dfs

[    f1_score  precision  recall  f1_score_rel  precision_rel  recall_rel  \
 0   0.000000        0.0     0.0           0.0            0.0         0.0   
 1   0.000000        0.0     0.0           0.0            0.0         0.0   
 2   1.000000        1.0     1.0           1.0            1.0         1.0   
 3   1.000000        1.0     1.0           1.0            1.0         1.0   
 4   0.000000        0.0     0.0           0.0            0.0         0.0   
 ..       ...        ...     ...           ...            ...         ...   
 58  0.000000        0.0     0.0           0.0            0.0         0.0   
 59  0.666667        1.0     0.5           0.0            0.0         0.0   
 60  0.000000        0.0     0.0           0.0            0.0         0.0   
 61  0.000000        0.0     0.0           0.0            0.0         0.0   
 62  0.000000        0.0     0.0           0.0            0.0         0.0   
 
      ged  normed_ged                                           response  

In [44]:

def graph_from_erl(erl: qg.EnrichedEntitiesRelations):
    G = qg.nx.DiGraph()
    for node in erl.entities:
        G.add_node(node.identifier, label=node.type)
    for link in erl.relations:
        G.add_edge(
            link.entity,
            link.target,
            weight=link.link.instance_count,
            label=link.relation,
        )
    return G

In [45]:
for f, df in zip(files, dfs):
    # data=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    # prompt_type=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    model = re.findall(r"results/eval_results_\w+_(.+)\.csv", f)[0]
    # df['prompt_type'] = prompt_type
    df["model"] = (
        model.replace("_", " ")
        .replace("-", " ")
        .rstrip("-GGUF")
        .lstrip("NousResearch Hermes 3 Llama")
        .lstrip(".2")
        .lstrip(".3")
    )

df = pd.concat(dfs)
df

,f1_score,precision,recall,f1_score_rel,precision_rel,recall_rel,ged,normed_ged,response,model,cfg_name,zeroshot,stage,erl,generator,n_nodes,seed,erl_loaded
0,0.000000,0.0,0.0,0.000000,0.0,0.00,5.0,0.000000,The patient is undergoing a protocol event and...,3B,BTO,False,raw,"{""relations"":[{""entity"":""Patient"",""relation"":""...",llama,3,0,"relations=[EnrichedRelation(entity='Patient', ..."
1,0.000000,0.0,0.0,0.000000,0.0,0.00,5.0,0.000000,The patient is undergoing a protocol event and...,3B,BTO,False,constrained,"{""relations"":[{""entity"":""Patient"",""relation"":""...",llama,3,0,"relations=[EnrichedRelation(entity='Patient', ..."
2,1.000000,1.0,1.0,1.000000,1.0,1.00,0.0,1.000000,The patient is undergoing a protocol event and...,3B,BTO,False,aligned,"{""relations"":[{""entity"":""Patient"",""relation"":""...",llama,3,0,"relations=[EnrichedRelation(entity='Patient', ..."
3,1.000000,1.0,1.0,1.000000,1.0,1.00,0.0,1.000000,"a Patient has a undergo with Protocol Event, a...",3B,BTO,False,raw,"{""relations"":[{""entity"":""Patient"",""relation"":""...",templated,3,0,"relations=[EnrichedRelation(entity='Patient', ..."
4,0.000000,0.0,0.0,0.000000,0.0,0.00,5.0,0.000000,"a Patient has a undergo with Protocol Event, a...",3B,BTO,False,constrained,"{""relations"":[{""entity"":""Patient"",""relation"":""...",templated,3,0,"relations=[EnrichedRelation(entity='Patient', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,0.000000,0.0,0.0,0.000000,0.0,0.00,9.0,0.000000,"A person, which edited a movie and attended a ...",3B,DBpedia,False,constrained,"{""relations"":[{""entity"":""person"",""relation"":""c...",human,5,121,"relations=[EnrichedRelation(entity='person', r..."
14,0.888889,1.0,0.8,0.750000,1.0,0.60,3.0,0.666667,"A person, which edited a movie and attended a ...",3B,DBpedia,False,aligned,"{""relations"":[{""entity"":""person"",""relation"":""c...",human,5,121,"relations=[EnrichedRelation(entity='person', r..."
15,1.000000,1.0,1.0,0.000000,0.0,0.00,8.0,0.111111,A person which edited a movie and is the auteu...,3B,DBpedia,False,raw,"{""relations"":[{""entity"":""coach"",""relation"":""al...",human,5,73,"relations=[EnrichedRelation(entity='coach', re..."
16,0.000000,0.0,0.0,0.000000,0.0,0.00,14.0,-0.272727,A person which edited a movie and is the auteu...,3B,DBpedia,False,constrained,"{""relations"":[{""entity"":""coach"",""relation"":""al...",human,5,73,"relations=[EnrichedRelation(entity='coach', re..."


In [46]:
df

,f1_score,precision,recall,f1_score_rel,precision_rel,recall_rel,ged,normed_ged,response,model,cfg_name,zeroshot,stage,erl,generator,n_nodes,seed,erl_loaded
0,0.000000,0.0,0.0,0.000000,0.0,0.00,5.0,0.000000,The patient is undergoing a protocol event and...,3B,BTO,False,raw,"{""relations"":[{""entity"":""Patient"",""relation"":""...",llama,3,0,"relations=[EnrichedRelation(entity='Patient', ..."
1,0.000000,0.0,0.0,0.000000,0.0,0.00,5.0,0.000000,The patient is undergoing a protocol event and...,3B,BTO,False,constrained,"{""relations"":[{""entity"":""Patient"",""relation"":""...",llama,3,0,"relations=[EnrichedRelation(entity='Patient', ..."
2,1.000000,1.0,1.0,1.000000,1.0,1.00,0.0,1.000000,The patient is undergoing a protocol event and...,3B,BTO,False,aligned,"{""relations"":[{""entity"":""Patient"",""relation"":""...",llama,3,0,"relations=[EnrichedRelation(entity='Patient', ..."
3,1.000000,1.0,1.0,1.000000,1.0,1.00,0.0,1.000000,"a Patient has a undergo with Protocol Event, a...",3B,BTO,False,raw,"{""relations"":[{""entity"":""Patient"",""relation"":""...",templated,3,0,"relations=[EnrichedRelation(entity='Patient', ..."
4,0.000000,0.0,0.0,0.000000,0.0,0.00,5.0,0.000000,"a Patient has a undergo with Protocol Event, a...",3B,BTO,False,constrained,"{""relations"":[{""entity"":""Patient"",""relation"":""...",templated,3,0,"relations=[EnrichedRelation(entity='Patient', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,0.000000,0.0,0.0,0.000000,0.0,0.00,9.0,0.000000,"A person, which edited a movie and attended a ...",3B,DBpedia,False,constrained,"{""relations"":[{""entity"":""person"",""relation"":""c...",human,5,121,"relations=[EnrichedRelation(entity='person', r..."
14,0.888889,1.0,0.8,0.750000,1.0,0.60,3.0,0.666667,"A person, which edited a movie and attended a ...",3B,DBpedia,False,aligned,"{""relations"":[{""entity"":""person"",""relation"":""c...",human,5,121,"relations=[EnrichedRelation(entity='person', r..."
15,1.000000,1.0,1.0,0.000000,0.0,0.00,8.0,0.111111,A person which edited a movie and is the auteu...,3B,DBpedia,False,raw,"{""relations"":[{""entity"":""coach"",""relation"":""al...",human,5,73,"relations=[EnrichedRelation(entity='coach', re..."
16,0.000000,0.0,0.0,0.000000,0.0,0.00,14.0,-0.272727,A person which edited a movie and is the auteu...,3B,DBpedia,False,constrained,"{""relations"":[{""entity"":""coach"",""relation"":""al...",human,5,73,"relations=[EnrichedRelation(entity='coach', re..."


In [47]:
df["prompt_type"] = df["zeroshot"].apply(lambda x: "zero shot" if x else "one shot")
df.drop(columns=["zeroshot"], inplace=True)
df

,f1_score,precision,recall,f1_score_rel,precision_rel,recall_rel,ged,normed_ged,response,model,cfg_name,stage,erl,generator,n_nodes,seed,erl_loaded,prompt_type
0,0.000000,0.0,0.0,0.000000,0.0,0.00,5.0,0.000000,The patient is undergoing a protocol event and...,3B,BTO,raw,"{""relations"":[{""entity"":""Patient"",""relation"":""...",llama,3,0,"relations=[EnrichedRelation(entity='Patient', ...",one shot
1,0.000000,0.0,0.0,0.000000,0.0,0.00,5.0,0.000000,The patient is undergoing a protocol event and...,3B,BTO,constrained,"{""relations"":[{""entity"":""Patient"",""relation"":""...",llama,3,0,"relations=[EnrichedRelation(entity='Patient', ...",one shot
2,1.000000,1.0,1.0,1.000000,1.0,1.00,0.0,1.000000,The patient is undergoing a protocol event and...,3B,BTO,aligned,"{""relations"":[{""entity"":""Patient"",""relation"":""...",llama,3,0,"relations=[EnrichedRelation(entity='Patient', ...",one shot
3,1.000000,1.0,1.0,1.000000,1.0,1.00,0.0,1.000000,"a Patient has a undergo with Protocol Event, a...",3B,BTO,raw,"{""relations"":[{""entity"":""Patient"",""relation"":""...",templated,3,0,"relations=[EnrichedRelation(entity='Patient', ...",one shot
4,0.000000,0.0,0.0,0.000000,0.0,0.00,5.0,0.000000,"a Patient has a undergo with Protocol Event, a...",3B,BTO,constrained,"{""relations"":[{""entity"":""Patient"",""relation"":""...",templated,3,0,"relations=[EnrichedRelation(entity='Patient', ...",one shot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,0.000000,0.0,0.0,0.000000,0.0,0.00,9.0,0.000000,"A person, which edited a movie and attended a ...",3B,DBpedia,constrained,"{""relations"":[{""entity"":""person"",""relation"":""c...",human,5,121,"relations=[EnrichedRelation(entity='person', r...",one shot
14,0.888889,1.0,0.8,0.750000,1.0,0.60,3.0,0.666667,"A person, which edited a movie and attended a ...",3B,DBpedia,aligned,"{""relations"":[{""entity"":""person"",""relation"":""c...",human,5,121,"relations=[EnrichedRelation(entity='person', r...",one shot
15,1.000000,1.0,1.0,0.000000,0.0,0.00,8.0,0.111111,A person which edited a movie and is the auteu...,3B,DBpedia,raw,"{""relations"":[{""entity"":""coach"",""relation"":""al...",human,5,73,"relations=[EnrichedRelation(entity='coach', re...",one shot
16,0.000000,0.0,0.0,0.000000,0.0,0.00,14.0,-0.272727,A person which edited a movie and is the auteu...,3B,DBpedia,constrained,"{""relations"":[{""entity"":""coach"",""relation"":""al...",human,5,73,"relations=[EnrichedRelation(entity='coach', re...",one shot


In [48]:
df_clean = df.dropna()
df_clean.describe()

,f1_score,precision,recall,f1_score_rel,precision_rel,recall_rel,ged,normed_ged,n_nodes,seed
count,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000
mean,0.361611,0.375103,0.365021,0.365912,0.379630,0.362757,6.876543,0.259388,3.222222,24.333333
std,0.397997,0.412269,0.409129,0.457803,0.469559,0.459818,13.989624,0.348556,0.632456,43.732997
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.272727,3.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,3.000000,3.000000
50%,0.285714,0.333333,0.200000,0.000000,0.000000,0.000000,4.000000,0.090909,3.000000,6.000000
75%,0.666667,0.750000,0.800000,1.000000,1.000000,1.000000,5.000000,0.400000,3.000000,10.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,68.000000,1.000000,5.000000,157.000000


In [49]:
df["erl_loaded"] = df["erl"].apply(qg.EnrichedEntitiesRelations.model_validate_json)
# df["response_loaded"] = df["response"].apply(lambda x: qg.EnrichedEntitiesRelations.model_validate_json(x) if isinstance(x, str) else None)
df["erl_graph"] = df["erl_loaded"].apply(graph_from_erl)
# df["response_graph"] = df["response"].apply(lambda x: graph_from_erl if x is not None else None)

In [50]:
# df["normed_ged"] = 1 - df["ged"] / df["erl_graph"].apply(
#     lambda x: (len(x.nodes) + len(x.nodes))
# )

In [51]:
df_clean["generator"].unique(), df_clean["model"].unique()

(array(['llama', 'templated', 'human'], dtype=object),
 array([' 3B '], dtype=object))

In [52]:
means = df_clean.groupby(
    ["n_nodes", "prompt_type", "stage", "model", "generator", "cfg_name"]
)[["f1_score", "normed_ged"]].mean()
counts = df_clean.groupby(
    ["n_nodes", "stage", "prompt_type", "model", "generator", "cfg_name"]
)[["f1_score", "normed_ged"]].count()
means

f1_score  normed_ged
n_nodes prompt_type stage       model generator cfg_name                      
3       one shot    aligned     3B    human     DBpedia   0.633333    0.400000
                                      llama     BTO       0.478788    0.378788
                                      templated BTO       0.680476    0.483971
                    constrained 3B    human     DBpedia   0.000000    0.000000
                                      llama     BTO       0.000000    0.000000
                                      templated BTO       0.000000    0.000000
                    raw         3B    human     DBpedia   0.522222    0.244444
                                      llama     BTO       0.181818    0.109091
                                      templated BTO       0.700000    0.780000
5       one shot    aligned     3B    human     DBpedia   0.687831    0.215488
                    constrained 3B    human     DBpedia   0.000000   -0.127946
                    raw         3B    human     DBpedia   0.896296    0.269360

In [53]:
counts

f1_score  normed_ged
n_nodes stage       prompt_type model generator cfg_name                      
3       aligned     one shot    3B    human     DBpedia          3           3
                                      llama     BTO             11          11
                                      templated BTO             10          10
        constrained one shot    3B    human     DBpedia          3           3
                                      llama     BTO             11          11
                                      templated BTO             10          10
        raw         one shot    3B    human     DBpedia          3           3
                                      llama     BTO             11          11
                                      templated BTO             10          10
5       aligned     one shot    3B    human     DBpedia          3           3
        constrained one shot    3B    human     DBpedia          3           3
        raw         one shot    3B    human     DBpedia          3           3

In [54]:
column_map = {
    "f1_score": "$F_1$",
    "normed_ged": "$GED_{s}$",
    "stage": "Stage",
    "prompt_type": "Prompt Type",
    "model": "Version \\& Size",
    "n_nodes": "$k$",
    "generator": "Query Origin",
}

## Compare Query Origins on DBpedia with 8B model

In [55]:
query_gen_compare = df_clean[
    df_clean["model"].str.contains("3B")
    & df_clean["cfg_name"].str.contains("DBpedia")
    & df_clean["prompt_type"].str.contains("one shot")
]
grouped_query_gen_compare = query_gen_compare.groupby(
    ["n_nodes", "generator", "stage"]
)[["f1_score", "normed_ged"]].mean()
grouped_query_gen_compare = grouped_query_gen_compare.rename(
    columns=column_map
).rename_axis([column_map["n_nodes"], column_map["generator"], column_map["stage"]], axis=0)
with open("tables/compare_gen_query.tex", "w") as f:
    f.write(
        grouped_query_gen_compare.to_latex(
            float_format="%.2f",
            caption="Comparison of query generation methods for LLama 3.1 8B on DBpedia with one shot prompts.",
            label="tab:compare_gen_query",
        )
    )
grouped_query_gen_compare

$F_1$  $GED_{s}$
$k$ Query Origin Stage                           
3   human        aligned      0.633333   0.400000
                 constrained  0.000000   0.000000
                 raw          0.522222   0.244444
5   human        aligned      0.687831   0.215488
                 constrained  0.000000  -0.127946
                 raw          0.896296   0.269360

## Compare models across Datasets

In [56]:
datasets = [
    "DBpedia",
    # "UniProt",
    "BTO",
    # "DNB",
]
grouped_dfs = {}
for dataset in datasets:
    model_compare = df_clean[df_clean["cfg_name"].str.strip() == dataset]
    grouped_model_compare = model_compare.groupby(
        [
            "n_nodes",
            "stage",
            "prompt_type",
            "model",
            # "generator",
        ]
    )[["f1_score", "normed_ged"]].mean()
    grouped_model_compare = grouped_model_compare.rename(
        columns=column_map
    ).rename_axis(
        [
            column_map["n_nodes"],
            column_map["stage"],
            column_map["prompt_type"],
            column_map["model"],
            # column_map["generator"],
        ],
        axis=0,
    )
    grouped_dfs[dataset] = grouped_model_compare
    with open(f"tables/compare_model_{dataset}.tex", "w") as f:
        f.write(
            grouped_model_compare.to_latex(
                float_format="%.2f",
                caption=f"Comparison of models for {dataset}",
                label=f"tab:compare_model_{dataset}",
                # sparsify=False
            )
        )
dataset_keys = list(grouped_dfs.keys())
merged_grouped: pd.DataFrame = grouped_dfs[dataset_keys[0]].copy()
merged_grouped.rename(
    columns={
        "$F_1$": f"$F_1$_{dataset_keys[0]}",
        "$GED_{s}$": f"$GED_{{s}}$_{dataset_keys[0]}",
    },
    inplace=True,
)
for dataset, grouped_df in list(grouped_dfs.items())[1:]:
    print(f"Merging {dataset} into the grouped DataFrame")
    merged_grouped = pd.merge(
        merged_grouped,
        grouped_dfs[dataset].rename(
            columns={
                "$F_1$": f"$F_1$_{dataset}",
                "$GED_{s}$": f"$GED_{{s}}$_{dataset}",
            }
        ),
        on=[
            "$k$",
            "Stage",
            "Prompt Type",
            "Version \\& Size",
            # "Query Origin",
        ],
        # suffixes=(None, f"_{dataset}"),
    )
rename_map = {f"$F_1$_{dataset}": f"{dataset} $F_1$" for dataset in datasets} | {
    f"$GED_{{s}}$_{dataset}": f"{dataset} $GED_{{s}}$" for dataset in datasets
}
merged_grouped.rename(
    columns=rename_map,
    inplace=True,
)
with open("tables/compare_model_merged.tex", "w") as f:
    f.write(
        merged_grouped.to_latex(
            float_format="%.2f",
            caption="Comparison of models for DBpedia and UniProt",
            label="tab:compare_model_merged",
        )
    )

Merging BTO into the grouped DataFrame


In [57]:
grouped_dfs

{'DBpedia':                                                 $F_1$  $GED_{s}$
 $k$ Stage       Prompt Type Version \& Size                     
 3   aligned     one shot    3B               0.633333   0.400000
     constrained one shot    3B               0.000000   0.000000
     raw         one shot    3B               0.522222   0.244444
 5   aligned     one shot    3B               0.687831   0.215488
     constrained one shot    3B               0.000000  -0.127946
     raw         one shot    3B               0.896296   0.269360,
 'BTO':                                                 $F_1$  $GED_{s}$
 $k$ Stage       Prompt Type Version \& Size                     
 3   aligned     one shot    3B               0.574830   0.428875
     constrained one shot    3B               0.000000   0.000000
     raw         one shot    3B               0.428571   0.428571}

In [58]:
merged_grouped

DBpedia $F_1$  DBpedia $GED_{s}$  \
$k$ Stage       Prompt Type Version \& Size                                     
3   aligned     one shot    3B                    0.633333           0.400000   
    constrained one shot    3B                    0.000000           0.000000   
    raw         one shot    3B                    0.522222           0.244444   

                                             BTO $F_1$  BTO $GED_{s}$  
$k$ Stage       Prompt Type Version \& Size                            
3   aligned     one shot    3B                0.574830       0.428875  
    constrained one shot    3B                0.000000       0.000000  
    raw         one shot    3B                0.428571       0.428571

In [59]:
df_clean["cfg_name"].str.strip() == "DBpedia"

0     False
1     False
2     False
3     False
4     False
      ...  
13     True
14     True
15     True
16     True
17     True
Name: cfg_name, Length: 81, dtype: bool